In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mne
import numpy as np
from prep_function import (
    load_muse_data, base_filtering,
    median_filter_artifact_removal, dynamic_threshold_artifact_removal,
    auto_artifact_rejection, run_ica, annotate_ica_artifacts
)
from prep_output import (
    plot_annotated_eeg, extract_brainwaves,
    export_cleaned_data, plot_global_brainwaves,
    verify_against_reference
)
import os

In [12]:
csv_path = r"C:\Users\shakeda\Downloads\mindMonitor_2025-05-23--21-30-11.csv"
output_dir = "outputs"

In [3]:
# raw data, nan values interpolated
raw, timestamp_list = load_muse_data(csv_path)

precentage of NaN values in EEG data: 5.42%
Creating RawArray with float64 data, n_channels=4, n_times=41201
    Range : 0 ... 41200 =      0.000 ...   160.938 secs
Ready.


c:\Users\shakeda\Documents\Masters\Python\Muse-S-EEG-prep\prep_function.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean = (df_eeg.interpolate(method='linear', axis=0).fillna(method='bfill').fillna(method='ffill'))


In [4]:
raw.plot()

Using qt as 2D backend.


Channels marked as bad:
none


In [7]:
base_filtering(raw)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 845 samples (3.301 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples

<RawArray | 4 x 41201 (160.9 s), ~1.3 MB, data loaded>

In [9]:
raw.plot()

Channels marked as bad:
none


In [10]:
raw = median_filter_artifact_removal(raw)
raw = dynamic_threshold_artifact_removal(raw)
raw = auto_artifact_rejection(raw)

Finding segments below or above PTP threshold.


In [13]:
plot_annotated_eeg(raw, output_dir)

In [ ]:
# Call ICA

In [ ]:
brainwave_df = extract_brainwaves(raw, timestamp_list)

In [ ]:
# 1. Make sure your brainwave_df lines up with raw:
#    number of samples must match
assert brainwave_df.shape[0] == raw.n_times

# 2. Create a RawArray for the new “component” channels
comp_ch_names = list(brainwave_df.columns)
# treat these as miscellaneous channels
comp_ch_types = ['misc'] * len(comp_ch_names)

info_comp = mne.create_info(ch_names=comp_ch_names,
                            sfreq=raw.info['sfreq'],
                            ch_types=comp_ch_types)

data_comp = brainwave_df.values.T  # shape: (n_components, n_times)
raw_comp  = mne.io.RawArray(data_comp, info_comp)

# 3. Copy your original raw and add the component‐RawArray
raw_combined = raw.copy().add_channels([raw_comp],
                                       force_update_info=True)

# 4. (Optional) Inspect
print(raw_combined.info)
# raw_combined.plot(n_channels=raw_combined.info['nchan'])
